# Run Vertex AI Matching Engine for Prediction

### Overview

This example demonstrates how to call an existing GCP ANN Service for online prediction. 

In [1]:
import os
import sys
PROJECT_ID = "uki-mlops-dev-demo"
VPC_NETWORK = "mlops-vpc"  
BUCKET_URI = "gs://uki-mlops-dev-demo-bucket"  
REGION = "us-central1" 
NUM_NEIGHBOURS = 10

In [45]:
import h5py

h5 = h5py.File("glove-100-angular.hdf5", "r")
test = h5["test"]

In [7]:
test

<HDF5 dataset "test": shape (10000, 100), type "<f4">

In [8]:
test[0]

array([ 0.39553  ,  0.23048  ,  0.82722  ,  0.10453  , -0.69281  ,
       -0.83357  , -0.49049  , -0.036362 , -0.48396  , -0.44315  ,
       -0.37407  , -0.13825  ,  0.3158   ,  0.16467  ,  0.1318   ,
       -0.34739  ,  0.30084  ,  0.26194  ,  0.60956  , -0.21171  ,
        0.26935  , -0.56669  ,  0.34927  ,  0.34816  , -0.014743 ,
        0.97688  ,  0.17702  ,  0.16185  ,  0.044074 , -0.68819  ,
        0.18073  ,  0.26355  ,  0.36275  , -0.73523  ,  0.39962  ,
        0.0037411, -0.15352  ,  0.10079  , -0.23187  , -0.7068   ,
        0.32768  , -0.012518 ,  0.038887 ,  0.67385  , -1.1839   ,
        0.91321  , -0.0060804,  0.026679 ,  0.42256  , -0.10934  ,
       -0.25663  , -0.22761  ,  0.34171  , -0.47256  , -0.075018 ,
       -0.55013  ,  0.5073   ,  0.096439 , -0.14561  ,  0.21227  ,
       -0.82953  ,  0.33062  ,  0.064787 ,  0.106    , -0.25982  ,
        0.24861  ,  0.2334   ,  0.45757  , -0.38603  , -0.19482  ,
       -0.83137  , -0.097219 , -0.23189  ,  0.21918  , -0.6416

#### Call ANN Index for Prediction

In [10]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

#### Call Online Queries  

Test your query:

In [56]:
INDEX_ENDPOINT = 'projects/106131389347/locations/us-central1/indexEndpoints/7753210641308975104'
DEPLOYED_INDEX_ID = 'ann_glove_deployed_nntoqo8r'
DEPLOYED_BRUTE_FORCE_INDEX_ID = 'brute_force_glove_deployed_nntoqo8r'

In [57]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_name=INDEX_ENDPOINT)

In [28]:
# Test query
response = my_index_endpoint.match(
    deployed_index_id=DEPLOYED_INDEX_ID, queries=test[:1], num_neighbors=NUM_NEIGHBOURS
)

response

[[MatchNeighbor(id='899605', distance=20.079139709472656),
  MatchNeighbor(id='1093903', distance=17.839481353759766),
  MatchNeighbor(id='296543', distance=16.99886703491211),
  MatchNeighbor(id='21495', distance=16.8770694732666),
  MatchNeighbor(id='689839', distance=16.852933883666992),
  MatchNeighbor(id='702494', distance=16.84667205810547),
  MatchNeighbor(id='518781', distance=16.65009307861328),
  MatchNeighbor(id='505832', distance=16.334247589111328),
  MatchNeighbor(id='405251', distance=16.278234481811523),
  MatchNeighbor(id='1142011', distance=16.186227798461914)]]

In [52]:
my_index_endpoint.mutate_deployed_index(
    deployed_index_id= DEPLOYED_INDEX_ID,
    min_replica_count= 1,
    max_replica_count= 1
)

Mutating index MatchingEngineIndexEndpoint index_endpoint: projects/106131389347/locations/us-central1/indexEndpoints/7753210641308975104
Mutate index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/106131389347/locations/us-central1/indexEndpoints/7753210641308975104/operations/5475358243200958464
MatchingEngineIndexEndpoint index_endpoint Mutated. Resource name: projects/106131389347/locations/us-central1/indexEndpoints/7753210641308975104


resource name: projects/106131389347/locations/us-central1/indexEndpoints/7753210641308975104

In [54]:
my_index_endpoint.mutate_deployed_index(
    deployed_index_id= DEPLOYED_BRUTE_FORCE_INDEX_ID,
    min_replica_count= 1,
    max_replica_count= 1
)

Mutating index MatchingEngineIndexEndpoint index_endpoint: projects/106131389347/locations/us-central1/indexEndpoints/7753210641308975104
Mutate index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/106131389347/locations/us-central1/indexEndpoints/7753210641308975104/operations/4937178087730184192
MatchingEngineIndexEndpoint index_endpoint Mutated. Resource name: projects/106131389347/locations/us-central1/indexEndpoints/7753210641308975104


resource name: projects/106131389347/locations/us-central1/indexEndpoints/7753210641308975104

### Compute Recall

Use the deployed brute force Index as the ground truth to calculate the recall of ANN Index. Note that you can run multiple queries in a single match call. </br> 
#### The test on 10,000 datapoints takes approximately 3 seconds for ScaNN search, 36 seconds for Brute Force search. 

In [53]:
# Retrieve nearest neighbors for both the tree-AH index and the brute-force index
tree_ah_response_test = my_index_endpoint.match(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=list(test),
    num_neighbors=NUM_NEIGHBOURS,
)

In [55]:
brute_force_response_test = my_index_endpoint.match(
    deployed_index_id=DEPLOYED_BRUTE_FORCE_INDEX_ID,
    queries=list(test),
    num_neighbors=NUM_NEIGHBOURS,
)

In [44]:
# Calculate recall by determining how many neighbors were correctly retrieved as compared to the brute-force option.
recalled_neighbors = 0
for tree_ah_neighbors, brute_force_neighbors in zip(
    tree_ah_response_test, brute_force_response_test
):
    tree_ah_neighbor_ids = [neighbor.id for neighbor in tree_ah_neighbors]
    brute_force_neighbor_ids = [neighbor.id for neighbor in brute_force_neighbors]

    recalled_neighbors += len(
        set(tree_ah_neighbor_ids).intersection(brute_force_neighbor_ids)
    )

recall = recalled_neighbors / len(
    [neighbor for neighbors in brute_force_response_test for neighbor in neighbors]
)

print("Recall: {}".format(recall))

Recall: 0.58067


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.
You can also manually delete resources that you created by running the following code.

In [42]:
# Force undeployment of indexes and delete endpoint
INDEX_ENDPOINT = 'projects/106131389347/locations/us-central1/indexEndpoints/32492767624036352'
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_name=INDEX_ENDPOINT)
#my_index_endpoint.delete(force=True)

Deleting MatchingEngineIndexEndpoint : projects/106131389347/locations/us-central1/indexEndpoints/32492767624036352
Delete MatchingEngineIndexEndpoint  backing LRO: projects/106131389347/locations/us-central1/operations/2285683807115804672
MatchingEngineIndexEndpoint deleted. . Resource name: projects/106131389347/locations/us-central1/indexEndpoints/32492767624036352


In [ ]:
# Delete indexes
tree_ah_index.delete()
brute_force_index.delete()

In [58]:
my_index_endpoint.undeploy_all(True)

Undeploying MatchingEngineIndexEndpoint index_endpoint: projects/106131389347/locations/us-central1/indexEndpoints/7753210641308975104
Undeploy MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/106131389347/locations/us-central1/indexEndpoints/7753210641308975104/operations/3266624100952440832
MatchingEngineIndexEndpoint index_endpoint undeployed. Resource name: projects/106131389347/locations/us-central1/indexEndpoints/7753210641308975104
Undeploying MatchingEngineIndexEndpoint index_endpoint: projects/106131389347/locations/us-central1/indexEndpoints/7753210641308975104
Undeploy MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/106131389347/locations/us-central1/indexEndpoints/7753210641308975104/operations/928692944393732096
MatchingEngineIndexEndpoint index_endpoint undeployed. Resource name: projects/106131389347/locations/us-central1/indexEndpoints/7753210641308975104


resource name: projects/106131389347/locations/us-central1/indexEndpoints/7753210641308975104